## Artificial Neural Network for Customer Churn Prediction
[Notebook from: http://bit.ly/byte-size-data-science]<br/>

In [ ]:
import sys
import types
import pandas as pd
import io
import requests

url = 'https://github.com/jacquesroy/byte-size-data-science/raw/master/data/customer_churn.csv'
content = requests.get(url).content
dataset = pd.read_csv(io.StringIO(content.decode('utf-8')))
dataset.head()


In [ ]:
import numpy as np

X = dataset.iloc[:,2:17].values # Columns from Gender on
y = dataset.iloc[:,1].values # CHURN column

### Encoding:
- Categorical: Gender, Status, Car Owner, Paymethod, LocalBilltype, LongDistanceBilltype

Other encoding could be used for some attributes. For example `onehotencoder` for Gender and Status

In [ ]:
# Encoding categorical data before split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
# columns: 0 (Gender), 1(Status), 4 (Car owner), 10 (Payment method), 11 (LocalBillType), 12 (LongDistanceBillType)

labelencoder_X_0 = LabelEncoder()
X[:,0] = labelencoder_X_0.fit_transform(X[:,0])
labelencoder_X_1 = LabelEncoder()
X[:,1] = labelencoder_X_1.fit_transform(X[:,1])
labelencoder_X_4 = LabelEncoder()
X[:,4] = labelencoder_X_4.fit_transform(X[:,4])
labelencoder_X_10 = LabelEncoder()
X[:,10] = labelencoder_X_10.fit_transform(X[:,10])
labelencoder_X_11 = LabelEncoder()
X[:,11] = labelencoder_X_11.fit_transform(X[:,11])
labelencoder_X_12 = LabelEncoder()
X[:,12] = labelencoder_X_12.fit_transform(X[:,12])

In [ ]:
# Since the answer column is "T" or "F", we have to encode it
labelencoder_y = LabelEncoder()
y = labelencoder_y.fit_transform(y)

# Let's see what the transformation did
print(y)

In [ ]:
# Split dataset
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)
X_train[0]

In [ ]:
# Feature scaling
# we need all the values in a standardized range 
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

# Look at the values in one record
X_train[0]

In [ ]:
# see: https://keras.io
import keras
from keras.models import Sequential # to initialize NN
from keras.layers import Dense 

In [ ]:
# Create a sequential model (most common in Keras)
classifier = Sequential()

# Create the first hidden layer
classifier.add(Dense(8, activation='relu', input_shape=(15,))) # there are 14 attributes

# Create the second hidden layer
classifier.add(Dense(8, activation='relu'))

# Create the output layer
classifier.add(Dense(1, activation='sigmoid'))

In [ ]:
# Compile the neural network
classifier.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
# Train the model (fit)
# batch size: number of records used in each epoch
classifier.fit(X_train, y_train, batch_size=18, epochs=100)

In [ ]:
# Testing
y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)

In [ ]:
# confusion matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

## Meaning
- True — True (top left) : Prediction of True correct
- True — False (top right): Predicted True while False
- False — True (bottom left): Predicted False while True
- False — False (bottom right) : Prediction of False correct


In [ ]:
# accuracy: number of correct predictions divided by the total number of predictions
accuracy = (cm[0,0] + cm[1,1]) / (cm[0,0] + cm[0,1] + cm[1,0] + cm[1,1])
print(accuracy)

In [ ]:
# Precision
precision = cm[0,0] / (cm[0,0] + cm[0,1])
print(precision)

In [ ]:
# Recall
recall = cm[0,0] / (cm[0,0] + cm[1,0])
print(recall)